In [112]:

import pandas as pd
import colorcet as cc
import polyline
import math
import numpy as np
colour_palette = cc.CET_C6s

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, reset_output, output_file

output_notebook()

### making data a bit easier to see
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### should be outside the repo
### edit this to match your local environment
# personal computer
data_location = r'/Users/jj/code/strava_data/'

# work computer
#data_location = r'/Users/jonathan.jackson/Code/strava_data/'

Loading BokehJS ...

In [108]:
### will return a list of colours of n length from the colour_palette
def get_colours(n):
    distance = int(len(colour_palette) / n)
    return [x for i, x in enumerate(colour_palette) if i % distance == 0]

### will assign a colour to each distinct value in a categorical col
def create_colour_map(s):
    unique = np.sort(s.unique())
    colours = get_colours(len(unique))
    return dict(zip(unique, colours))

In [109]:
### Load the data
raw = pd.read_csv(data_location+'all_activities_cleaned.csv', index_col=0, parse_dates=True)
raw = raw.apply(lambda x: pd.to_datetime(x) if 'date' in x.name else x)

In [113]:
### Convert lat and long coordinates to mercator
def geographic_to_web_mercator(x_lon, y_lat):     
    if abs(x_lon) <= 180 and abs(y_lat) < 90:          
        num = x_lon * 0.017453292519943295         
        x = 6378137.0 * num         
        a = y_lat * 0.017453292519943295          
        x_mercator = x         
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))         
        return x_mercator, y_mercator      
    else:         
        print('Invalid coordinate values for conversion')

### convert the strava map data format to a polyline
map_data = raw.copy()
map_data = map_data.dropna(subset=['map_summary_polyline'])
map_data['polyline'] = map_data['map_summary_polyline'].apply(polyline.decode)
map_data['mercator'] = map_data['polyline'].apply(lambda x: [geographic_to_web_mercator(y[1], y[0]) for y in x])

### mapping my own bikes to different ride types
def custom_activity_type(a):
    if a['bike_name'] in ['Transition Smuggler', 'Canyon Spectral', 'Fat Bike']: return 'Mountain Bike Ride'
    elif a['bike_name'] in ['Avanti Giro 2', 'Cervelo Aspero']: return 'Road/Gravel Ride'
    elif a['type'] == 'Bike': return 'Ride Unknown'
    elif a['type'] == 'Run': return 'Run'
    elif a['type'] in ['Walk', 'Hike']: return 'Walk'
    else: return a['type']

map_data['activity_group'] = map_data.apply(custom_activity_type, axis=1)

In [111]:
def categorical_map(cat_col, activity_limit=None):
    ### only get rows where we have map data
    df = map_data.dropna(subset=[cat_col])[['mercator', cat_col]]

    ### create subset of data
    if activity_limit: df = df[0:activity_limit]

    ### create colour map
    df['colour'] = df[cat_col].map(create_colour_map(df[cat_col]))

    p = figure(width=800, height=800, x_axis_type="mercator", y_axis_type="mercator", tools="pan,wheel_zoom,reset", active_drag='pan', active_scroll='wheel_zoom')

    ### there are other options for a background map tile - check out here https://docs.bokeh.org/en/latest/docs/user_guide/topics/geo.html#geographical-data
    p.add_tile("CartoDB.Positron")

    ### one line per activity
    for activity in df.values:
        x = [a[0] for a in activity[0]]
        y = [a[1] for a in activity[0]]
        p.line(x, y, line_width=2, color=activity[2], line_alpha=.5, legend_label=activity[1])
    
    ### click the legend entries to show/hide activity types
    p.legend.click_policy="hide"

    ### we eilel save a version of the map and display it belpow
    reset_output()
    output_file(data_location+cat_col+'_map.html')
    output_notebook()
    
    show(p)

categorical_map('activity_group', 100)

Loading BokehJS ...